In [4]:
import ast
import re
from event_loop.preprocessing.dataframe import *

import metrics
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

%load_ext autoreload
%load_ext memory_profiler

# Prerequisuites

## Activity Action Model
Train a Model with the task of classifying Start, End and NoAction events in the interleaved data. 
Training is done during a "warmup" phase with generated training data. 

### Load Data

In [5]:
# HR data in data/Train/R1 is missing frame.number. We take another (already filtered) dataset and apply our feature extraction to this one
df_train_in = pd.read_csv('../../data/VALID/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

In [6]:
# This is the Interleaved Data Set for our pipeline
df_il_in = pd.read_csv('../../data/PTP-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

### Preprocessing Training Data

In [8]:
%autoreload 2
# data is at R1 Level. Apply filter and feature extraction
df_train = pre_process(df_train_in)
df_test = pre_process(df_il_in)


In [114]:
# Load start and end events from ground truth data.
# Tag according frames in interleaved data for testing
df_gt = pd.read_csv("../../data_v3/ptp_ground_truth.csv")

start_indices = df_gt["start"].tolist()
end_indices = df_gt["actual_end"].tolist()

df_test["ActivityAction"] = df_test["frame.number"].apply(lambda x: "Activity Start" if x in start_indices else
("Activity End" if x in end_indices else "NoAction"))

In [116]:
# ------------ OPTIONAL ---------------
# TODO Duplicate with Activity Model - move down and delete
# Form sequences in training data by grouping
df_train = df_train.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_train["SequenceNumber"] = df_train.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
df_train["SequenceNumber"] -= df_train['SequenceNumber'].min()

In [15]:
def mark_start_end(df):
    # Mark start event of each BusinessActivity Instance
    df["activityStart"] = df.groupby(["BusinessActivity", "InstanceNumber", ]).cumcount() == 0
    # Mark end event of each Business Activity Instance
    df["activityEnd"] = df.groupby(["BusinessActivity", "InstanceNumber", ]).cumcount(ascending=False) == 0
    # Merge start and end columns to form labels
    df["ActivityAction"] = df.apply(lambda row: "Activity Start" if row["activityStart"] else (
        "Activity End" if row["activityEnd"] else 'NoAction'), axis=1)

    return df.drop(["activityStart", 'activityEnd'], axis=1)


df_train = mark_start_end(df_train)

In [16]:
cols = ["event_with_roles", "request_method_call", "selective_file_data", 
        "origin_method","origin_file_data"]


In [17]:
def dict_to_features(dict):
    return [[{**d, "bias": 1.0}] for d in dict]


def extract_labels(labels):
    return [[y] for y in labels]

In [18]:
# exclude from training data 
df_train_filt = df_train[~df_train["SequenceNumber"].isin([128])]


In [19]:
df_train["InstanceNumber"].value_counts()

InstanceNumber
31    505
48    504
64    504
19    504
33    503
     ... 
44    265
39    214
59    214
24    214
36    212
Name: count, Length: 67, dtype: int64

In [20]:
train_features = df_train_filt[cols].to_dict("records")
train_features = dict_to_features(train_features)
train_labels = extract_labels(df_train_filt["ActivityAction"])

In [21]:
test_features = df_test[cols].to_dict("records")
test_features = dict_to_features(test_features)
test_labels = extract_labels(df_test["ActivityAction"])

### Model Training

In [22]:
# optional Train Test split for evaluation on training data
# In prod case, we train on 100% training data and evaluate on interleaved data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.3, random_state=42)

In [23]:
%%time
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
crf.fit(train_features, train_labels)

CPU times: user 1.24 s, sys: 19.8 ms, total: 1.26 s
Wall time: 2.2 s


CRF(all_possible_transitions=True, c1=0.1, c2=0.01, max_iterations=200)

In [24]:
%%time
import sklearn_crfsuite

crf_test = sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
crf_test.fit(X_train, y_train)

NameError: name 'evaluate' is not defined

### Optimization

In [25]:
from sklearn.metrics import make_scorer
import scipy
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import metrics

# define fixed parameters and parameters to search
crf2 = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=200, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='macro', labels=np.unique(test_labels))

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=5, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=150, 
                        scoring=f1_scorer)
#rs.fit(train_features, train_labels)

#crf = rs.best_estimator_

### Evaluation

In [26]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn_crfsuite import metrics


def flatten(xss):
    return [x for xs in xss for x in xs]


def evaluate(model, x, y_true):
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels=model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    [print(label, "\n", matrix) for matrix, label in
     zip(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred), labels=model.classes_), model.classes_)]


In [27]:
evaluate(crf, test_features, test_labels)

0.8432082382941252
                precision    recall  f1-score   support

Activity Start       1.00      1.00      1.00        63
      NoAction       0.99      1.00      0.99      3783
  Activity End       1.00      0.37      0.53        63

      accuracy                           0.99      3909
     macro avg       1.00      0.79      0.84      3909
  weighted avg       0.99      0.99      0.99      3909

Activity Start 
 [[3846    0]
 [   0   63]]
NoAction 
 [[  86   40]
 [   0 3783]]
Activity End 
 [[3846    0]
 [  40   23]]


In [28]:
pred = crf.predict(test_features)

In [29]:
pred_mg = crf.predict_marginals(test_features)

In [30]:
#margs = [pred_mg[i] for i in wrong_pred_idx]
columns = pred_mg[0][0].keys()
flat_margs = [[entry[column] for column in columns] for sublist in pred_mg for entry in sublist]
df_margs = pd.DataFrame(flat_margs, columns=columns)

In [31]:
#df_eval = pd.DataFrame([(pred[i], test_labels[i], df_test.iloc[i]["frame.number"] ) for i in wrong_pred_idx],columns = ["predicted","true","frame.number"],)

df_eval = pd.DataFrame({"predicted": pred, "true": test_labels, "frame.number":df_test["frame.number"], "file_data": df_test["file_data"], "evr": df_test["event_with_roles"]}).reset_index(drop=True)

df_eval = pd.concat([df_eval, df_margs], axis = 1)

df_eval["pred_true"] = df_eval["predicted"] == df_eval["true"]



In [32]:
from scipy.stats import entropy

entropy_cols = ['Activity Start', 'NoAction', 'Activity End']

# Calculate entropy for each row using the specified columns
df_eval["entropy"] = df_eval[entropy_cols].apply(entropy, axis=1)


In [33]:
df_eval[~ df_eval["pred_true"]].sort_values(by='entropy', ascending=False)

,predicted,true,frame.number,file_data,evr,Activity Start,NoAction,Activity End,pred_true,entropy
64,[NoAction],[Activity End],1887,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
2999,[NoAction],[Activity End],86895,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
2138,[NoAction],[Activity End],63000,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
154,[NoAction],[Activity End],5965,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
3477,[NoAction],[Activity End],100384,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
1657,[NoAction],[Activity End],49156,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
1249,[NoAction],[Activity End],37650,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
2244,[NoAction],[Activity End],65514,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
945,[NoAction],[Activity End],28958,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409
2884,[NoAction],[Activity End],83750,[1],Odoo Application->End Point (Procurement):[Htt...,0.000241,0.579766,0.419993,False,0.682409


In [34]:
eval_cols = ["event_with_roles","pgsql.query", "request_method_call", "selective_file_data", 
        "origin_method","origin_file_data"]

df_eval = df_eval.merge(df_test[["frame.number", *eval_cols]], how="left",left_on="frame.number", right_on="frame.number")

In [35]:
df_eval["true_first"] = df_eval["true"].apply(lambda x: x[0])

We observe a high entropy > 0.5 for all wrong classifications

-> Apply fallback model for this cases

## Activity Classifier

In [36]:
def sequence_by_activities(data, seq_data):
    return [data[seq_data == i] for i in range(seq_data.max())]

In [37]:
feature_cols = ["event_with_roles", "request_method_call", "selective_file_data", 
        "origin_method","origin_file_data"]


In [38]:
# List of dataframes each containing one activity sequence
train_activity_sequences = sequence_by_activities(df_train, df_train["SequenceNumber"])

In [39]:
# Sequences without window features

def dict_to_feature_sequence(dict):
    return [{**d, "bias": 1.0} for d in dict]

def df_to_features(df):
    return dict_to_feature_sequence(df.to_dict("records"))

train_features_seq = [df_to_features(df[feature_cols]) for df in train_activity_sequences]
train_labels_seq = [df["BusinessActivity"].values for df in train_activity_sequences]

In [40]:
# Single Events no window features 

def dict_to_feature(dict):
    return [[{**d, "bias": 1.0}] for d in dict]

def extract_labels(labels):
    return [[y] for y in labels]

train_features = dict_to_feature(df_train[feature_cols].to_dict("records"))
train_labels = extract_labels(df_train["BusinessActivity"])

In [41]:
# Single Events w. window features

# Apply sequencing - flatten later

def seq2features(seq, bw, fw): 
    return [event2features(seq, i, bw, fw) for i in range(len(seq))]

def event2features(seq, i, bw, fw):
    features = {"bias": 1.0}
    
    features.update({
        f"0:{k}": v for k,v in seq[i].items()
    })
    
    for j in range(1, bw+1): 
        index = i-j
        if index >= 0: 
            features.update({
                f"-{j}:{k}": v for k,v in seq[index].items()
            })
        else: 
            features.update({
                 f"-{j}:{k}": "NoMessage" for k,_ in seq[i].items()
            })
        
    for j in range(1,fw+1): 
        index = i + j
        if index < len(seq): 
             features.update({
                f"+{j}:{k}": v for k,v in seq[index].items()
            })
        else: 
            features.update({
                 f"+{j}:{k}": "NoMessage" for k,_ in seq[i].items()
            })
            
    return features

train_features_seq_window = [seq2features(seq[feature_cols].to_dict("records"), 10,10) for seq in train_activity_sequences]
train_labels_seq_window = [seq["BusinessActivity"] for seq in train_activity_sequences]

In [43]:
def flatten_and_encapsulate(list_of_list):
    return [[item] for sublist in list_of_list for item in sublist]

X_train = flatten_and_encapsulate(train_features_seq_window)
y_train = flatten_and_encapsulate(train_labels_seq_window)

In [115]:
%%time
import sklearn_crfsuite

activity_classifier= sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
activity_classifier.fit(X_train, y_train)

CPU times: user 10.5 s, sys: 215 ms, total: 10.7 s
Wall time: 11.8 s


CRF(all_possible_transitions=True, c1=0.1, c2=0.01, max_iterations=200)

In [46]:
def confidence_weighted_majority_voting(predictions):
    """
    Perform confidence-weighted majority voting on each sublist of predictions.

    :param predictions: A list of dictionaries where each dictionary contains predictions and their confidences.
    :return: A list of majority voted predictions for each sublist.
    """
    majority_voted_predictions = []
    for sublist in predictions:
        if not sublist:
            # If the sublist is empty, append None to the majority voted predictions
            majority_voted_predictions.append(None)
        else:
            # Initialize variables to store cumulative confidences for each prediction
            cumulative_confidences = {label: 0.0 for label in sublist[0].keys()}
            
            # Calculate cumulative confidences for each prediction across all dictionaries in the sublist
            for prediction_dict in sublist:
                for label, confidence in prediction_dict.items():
                    cumulative_confidences[label] += confidence
            
            # Find the prediction with the maximum cumulative confidence
            majority_voted_prediction = max(cumulative_confidences, key=cumulative_confidences.get)
            majority_voted_predictions.append(majority_voted_prediction)

    return majority_voted_predictions

## Activity Model
The activity model utilises multiple sliding windows over the training data for pattern matching



In [47]:
from sklearn.preprocessing import LabelEncoder
from numpy.lib.stride_tricks import sliding_window_view


def get_unique_sequences(seq_data):
    # Convert each array to a tuple and create a set of tuples
    array_set = set(tuple(arr) for arr in seq_data)

    # Convert the set of tuples back to a list of NumPy arrays
    return [np.array(arr) for arr in array_set]


df_train["joined"] = df_train["event_with_roles"] + df_train["selective_file_data"]

# Label Encode Training Data 
le = LabelEncoder()
df_train["joined_LE"] = le.fit_transform(df_train["joined"])

# Mark groups of Instance Number and BusinessActivity with sequence numbers
df_train = df_train.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_train["SequenceNumber"] = df_train.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
# Align Sequence Numbers so that they start at 0
df_train["SequenceNumber"] -= df_train['SequenceNumber'].min()

# Divides dataframe into arrays according to to Sequence Data Indicator
data_joined_LE = sequence_by_activities(df_train["joined_LE"], df_train["SequenceNumber"])

unique_sequences = get_unique_sequences(data_joined_LE)

print(f"Reduced the number of sequences from {len(data_joined_LE)} to {len(unique_sequences)} unique ones")

def get_activity_model_data(max_window_length):
    return [np.concatenate([sliding_window_view(seq, i) for seq in unique_sequences], axis=0) for i in
                       range(max_window_length)]
    

# form sliding window sequences of Size N for Training Data 
#activity_model_data = get_activity_model_data(4)

Reduced the number of sequences from 408 to 66 unique ones


# Action Loop

Main loop. Gets raw R1 data as input. 
Applies filtering, activity action and sequence classification

In [48]:
records = df_il_in.to_dict("records")

In [49]:
from event_loop.event import Event


def get_max_from_dict(d: dict):
    return max(d, key= lambda k: d[k])
    

def classify_event(event: Event): 
    margs = crf.predict_marginals_single([event.to_features()])[0]
    pred = get_max_from_dict(margs)
    
    e = entropy([p for p in margs.values()])
    
    #print(f"{event.frame_number} {pred} {margs[pred]:.3f} {e:.3f},")
    
    
    true_val = df_eval[df_eval["frame.number"] == event.frame_number]["true"].iloc[0][0]
    #print(true_val)
    
    # Change to entropy of prediction
    if e > ENTROPY_THRESHOLD: 
    #if pred != true_val:
        # If pred is wrong we have two options for "wrong classifications" 
        # 1 -> We have No Action predicted although the stack should end here 
        # Idea 1: If the stack did not change after N events, emit it. 
        
        # Mark the confidence on the event.
        event.confidence = False 


        # 2 -> We have End predicted although the stack should continue.        
        event.activity_action = pred
            
    else: 
        event.confidence = True
        event.activity_action = pred


In [50]:
from event_loop.stack import Stack


def search_stack_for_request_frame(frame_number):
    for index, stack in enumerate(stacks):
        if stack.contains_request_frame(frame_number): 
            return index
    return -1

def search_window_for_sequence(seq): 
    """
    Check for pattern matches with the training data and return the count
    :param seq: array_like
                sequence of events
            
    :return: number of occurences of seq in training data
    """
    return np.sum(np.all(activity_model_data[len(seq)] == seq, axis = 1))


def classify_by_train_sequences(event: Event,n : int, exclude_indices: list[int]): 
    # search for existing stacks in training data 
    sequences = [le.transform([ e.to_activity_model_string() for e in stack]+[event.to_activity_model_string()]) for stack in stacks]  
    
    # loop to max 2 elements down
    for i in range(n, 1, -1):   
        res = [search_window_for_sequence(seq[-i:]) if j not in exclude_indices else -1 for j,seq in enumerate(sequences)]
        
        max_res = max(res)
        max_res_count = res.count(max_res)
        idx = np.argmax(res)
        
        if max_res > 0: 
            #print("res:",res, max_res, max_res_count, "->", idx)
            return idx
        
    return -1

def search_stream_index(event: Event, exclude_indices: list[int]) -> int: 
    indices = [i for i,stack in enumerate(stacks) if stack.contains_stream_index(event.stream_index) and i not in exclude_indices]

    if len(indices) == 1: 
        return indices[0]
    else:
        return -1
    

def check_stack_attributes(stacks: list[Stack], event: Event, exclude_indices: list[int]) -> int:
    for key, value in event.attributes.items():
        if key in PTP_ATTRIBUTES and value:
            indices = [i for i, stack in enumerate(stacks) if stack.contains_attribute(key,value) and i not in exclude_indices]
            print()
            
            if len(indices) ==1: 
                print("MATCH", indices)
                # we have a clear match -> return idx
                return indices[0]

    return -1

def check_stack_attributes_case_id(stacks: list[Stack], event: Event, exclude_indices: list[int]) -> int:
    for key, value in event.attributes.items():
        if key in PTP_ATTRIBUTES and value:
            indices = [i for i, stack in enumerate(stacks) if stack.case_id == Stack.case_id_from_attribute(key, value)]
            
            if len(indices) ==1: 
                # we have a clear match -> return idx
                return indices[0]

    return -1


def exclude_stacks_by_attribute(stacks: list[Stack], event: Event, stacks_out: list[Stack]) -> list[int]: 
    
    exclude_indices = []

        
    for key, value in event.attributes.items():
        if key in PTP_ATTRIBUTES and value:
            # exlucde all stacks that have a different attribute 
            exclude_indices.extend(i for i, stack in enumerate(stacks) if stack.has_attribute(key) and not stack.contains_attribute(key, value))
    return exclude_indices

def exclude_stacks_by_attribute_case_id(stacks: list[Stack], event: Event, stacks_out: list[Stack]) -> list[int]: 
    
    exclude_indices = []

    for key, value in event.attributes.items():
        if key in PTP_ATTRIBUTES and value:
            # exclude all stacks that have a different attribute 
            # 
            for i, stack in enumerate(stacks): 
                event_case_id = Stack.case_id_from_attribute(key, value)
                if event_case_id and stack.case_id and event_case_id != stack.case_id: 
                    exclude_indices.append(i)
            #exclude_indices.extend(i for i, stack in enumerate(stacks) if Stack.case_id_from_attribute(key, value) != stack.case_id and stack.case_id)
    return exclude_indices
    
    

In [112]:
%autoreload 2

import time
from event_loop.preprocessing.event import keep_event

# Parameter
EVENT_LOOP_CUTOFF_NO_ACTION = 3
EVENT_LOOP_CUTOFF_END_EVENT = 3
ENTROPY_THRESHOLD = 0.4 #0.5
MAX_WINDOW_SIZE = 10
VERBOSE = False
SETTING = "PTP"

# init variables
event_buffer: list[Event] = []
attribute_buffer: list[dict] = []
stacks: list[Stack] = []
stacks_out: list[Stack] = []
event_loop_index = 0


HR_ATTRIBUTES = ["applicant_id", "activity_id"]
PTP_ATTRIBUTES = ["sale_order_id", "sale_order_line_id","purchase_requisition_id","purchase_requisition_line_id",]


activity_model_data = get_activity_model_data(MAX_WINDOW_SIZE)
processing_times = []
processing_times_filter = []
buffer_sizes = []


for i, event_data in enumerate(records):
    start_time = time.time()

    buffer_sizes.append(sum([len(stack) for stack in stacks]))
    # Filter Event Stream
    if not keep_event(event_data):
        end_time = time.time()
        processing_times_filter.append(end_time - start_time)
        # skip event in loop
        continue
        
    
    
    # count every not filtered event for event loop index
    event_loop_index += 1

    # Extract Features and generate Event Object
    event = Event(event_data, event_loop_index, event_buffer, SETTING)
    event_buffer.append(event)
    
    classify_event(event)
    
    # Activity Action Classification
    activity_action = event.activity_action
    
    # Activity Matching
    if activity_action == "Activity Start": 
        stacks.append(Stack(SETTING,event))
        
    if activity_action == "NoAction": 
        if len(stacks) == 1: 
            stacks[0].append_event(event)
        elif event.origin_request_frame: 
            idx = search_stack_for_request_frame(event.origin_request_frame)
            stacks[idx].append_event(event)
        else: 
            # Check attributes of each stack
            
            # we can filter out stacks that already have attributes different to the event
            exclude_indices =  exclude_stacks_by_attribute(stacks, event, stacks_out)
    
            stack_index:int = check_stack_attributes(stacks, event, exclude_indices)
                    
            if stack_index == -1:        
                stack_index = classify_by_train_sequences(event, 4, exclude_indices)
            
            # for elements that are not matchable based on 2 sequences we fall back to stream index
            if stack_index == -1: 
                stack_index = search_stream_index(event, exclude_indices)    
            
            # fallback - no match add to first stack
            if stack_index == -1:
                res = next((i for i in range(len(stacks)) if i not in exclude_indices and stacks[i].confidence),-1)
                stack_index = res
                
            stacks[stack_index].append_event(event)
        
    if activity_action == "Activity End":
        
        stack_index = search_stack_for_request_frame(event.origin_request_frame)
        stacks[stack_index].append_event(event)
        
        if event.confidence: 
            if len(stacks) > 1: 
                stack = stacks.pop(stack_index)
                stacks_out.append(stack)
            else: 
                event.confidence = False
     

    # Loop through all currently open stacks
    for idx, stack in enumerate(stacks):
        last_event = stack[-1]
        # check for non-confident "No Action" Classifications. These could be "Activity End" Instead
        if not last_event.confidence and last_event.activity_action == "NoAction":
            # If a stack has not been continued for N event loops 
            if event_loop_index - last_event.event_loop_index > EVENT_LOOP_CUTOFF_NO_ACTION: 
                stacks.pop(idx)
                stacks_out.append(stack)
                
    for idx, stack in enumerate(stacks): 
        last_event = stack.events[-1]
        if not last_event.confidence and last_event.activity_action == "Activity End": 
            if event_loop_index - last_event.event_loop_index > EVENT_LOOP_CUTOFF_END_EVENT: 
            
                # we are now sure to pop the stack. 
                stacks.pop(idx)
                stacks_out.append(stack) 
                
    end_time = time.time()
    processing_times.append(end_time - start_time)
                
# pop all stacks that are still left
for stack in stacks: 
    stacks_out.append(stack)  


False 1887 NoAction 0.580 0.682 should be Activity End


MATCH [1]


MATCH [1]

MATCH [1]

MATCH [1]

MATCH [1]


MATCH [1]

MATCH [1]

MATCH [1]
False 5965 NoAction 0.580 0.682 should be Activity End

MATCH [0]

MATCH [0]



MATCH [1]


MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]


MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [2]

MATCH [0]

MATCH [0]

MATCH [0]


MATCH [1]


MATCH [1]

MATCH [1]

MATCH [1]

MATCH [1]


MATCH [1]

MATCH [1]

MATCH [1]
False 15058 NoAction 0.580 0.682 should be Activity End
False 15871 NoAction 0.853 0.418 should be Activity End
False 19296 NoAction 0.853 0.418 should be Activity End

MATCH [0]



MATCH [1]


MATCH [0]

MATCH [2]

MATCH [2]

MATCH [1]

MATCH [0]
False 20497 NoAction 0.580 0.682 should be Activity End

MATCH [1]

MATCH [1]

MATCH [1]


MA

In [113]:
import sys
import pickle
model_size = [
    sys.getsizeof(pickle.dumps(crf)),
    sys.getsizeof(pickle.dumps(activity_classifier)),
    sys.getsizeof(pickle.dumps(activity_model_data))
]

In [111]:
from statistics import mean,stdev
print("average processing time:",mean(processing_times)*1000)
print("max buffer size:", max(buffer_sizes),f"({max(buffer_sizes) / len(records) * 100})")

average processing time: 0.5412944658054479
max buffer size: 318 (0.141403720963324)


In [79]:
# classify stacks
# TODO Move into Model
def confidence_weighted_majority_voting(predictions):
    """
    Perform confidence-weighted majority voting on each sublist of predictions.

    :param predictions: A list of dictionaries where each dictionary contains predictions and their confidences.
    :return: A list of majority voted predictions for each sublist.
    """    

    # Initialize variables to store cumulative confidences for each prediction
    cumulative_confidences = {label: 0.0 for label in predictions[0][0].keys()}
    
    # Calculate cumulative confidences for each prediction across all dictionaries in the sublist
    for prediction_dict in predictions:
        for label, confidence in prediction_dict[0].items():
            cumulative_confidences[label] += confidence
    
    # Find the prediction with the maximum cumulative confidence
    return max(cumulative_confidences, key=cumulative_confidences.get)

 
 
 
    
def classify_stack(stack: Stack):
    seq = seq2features([event.to_features() for event in stack], 10,10)
    pred = activity_classifier.predict_marginals([[ele] for ele in seq])
    pred_cwmv = confidence_weighted_majority_voting(pred)
    return pred_cwmv
    
stack_predictions = [classify_stack(stack )for stack in stacks_out]

In [106]:
start = [stack[0].frame_number for stack in stacks_out]
end = [stack[-1].frame_number for stack in stacks_out]

res_df = pd.DataFrame({"start_pred":start, "end_pred":end})

eval_df = df_gt[["start", "actual_end"]].merge(res_df,how="left", left_on ="start", right_on = "start_pred").fillna(-1).astype(int)
eval_df["end_pred_true"] = eval_df["actual_end"] == eval_df["end_pred"]
eval_df["start_pred_true"] = eval_df["start"] == eval_df["start_pred"]
eval_df["start_end_true"] =eval_df["start_pred_true"] == eval_df["end_pred_true"]

display(eval_df)
print(f"Overall matching accuracy: {0.5 + eval_df['end_pred_true'].mean()/2}")

,start,actual_end,start_pred,end_pred,end_pred_true,start_pred_true,start_end_true
0,96,1322,96,1322,True,True,True
1,1367,1887,1367,1887,True,True,True
2,1940,2793,1940,2793,True,True,True
3,2818,15871,2818,11235,False,True,False
4,5563,5965,5563,5965,True,True,True
...,...,...,...,...,...,...,...
58,100724,104454,100724,108703,False,True,False
59,101210,105925,101210,106177,False,True,False
60,106266,108703,106266,112188,False,True,False
61,108727,109696,108727,109696,True,True,True


Overall matching accuracy: 0.746031746031746


In [104]:
df_aa_test = pd.DataFrame(df_test[["frame.number", "ActivityAction"]])
df_aa_test["ActivityAction"] = "NoAction"
df_aa_test.loc[df_aa_test["frame.number"].isin(eval_df["end_pred"]), "ActivityAction"] = "Activity End"
df_aa_test.loc[df_aa_test["frame.number"].isin(eval_df["start_pred"]), "ActivityAction"] = "Activity Start"
print(classification_report(test_labels, df_aa_test["ActivityAction"]))

                precision    recall  f1-score   support

  Activity End       0.70      0.70      0.70        63
Activity Start       1.00      1.00      1.00        63
      NoAction       0.99      0.99      0.99      3783

      accuracy                           0.99      3909
     macro avg       0.90      0.90      0.90      3909
  weighted avg       0.99      0.99      0.99      3909


In [100]:
# Function to check if intervals overlap
def intervals_overlap(row, df):
    overlapping_names = []
    overlapping_bps = set()
    for index, other_row in df.iterrows():
        if row.name != index and row['start'] <= other_row['actual_end'] and row['actual_end'] >= other_row['start']:
            overlapping_names.append(f"{other_row['activity_name']} {other_row['bp_id']}")
            overlapping_bps.add(other_row['bp_id'])
    return overlapping_names, list(overlapping_bps)

df_gt[["overlapping_activities", "overlapping_bps"]] = df_gt.apply(intervals_overlap, axis=1, df = df_gt, result_type="expand")

In [98]:
# Create dataframe with mapping of frame numbers to event stacks
frame_numbers = [event.frame_number for idx,stack in enumerate(stacks_out) for event in stack]
stack_numbers = [idx for idx,stack in enumerate(stacks_out) for event in stack]
case_id = [stack.case_id["id"]  if stack.case_id else -1 for idx, stack in enumerate(stacks_out) for event in stack]
sniff_time =  [event.sniff_time for idx,stack in enumerate(stacks_out) for event in stack]
sale_order_ids = [event.attributes["sale_order_id"] for idx,stack in enumerate(stacks_out) for event in stack]
sale_order_line_ids = [event.attributes["sale_order_line_id"] for idx,stack in enumerate(stacks_out) for event in stack]
purchase_requisition_ids = [event.attributes["purchase_requisition_id"] for idx,stack in enumerate(stacks_out) for event in stack]
purchase_requisition_line_ids = [event.attributes["purchase_requisition_line_id"] for idx,stack in enumerate(stacks_out) for event in stack]
purchase_order_ids = [event.attributes["purchase_order_id"] for idx,stack in enumerate(stacks_out) for event in stack]
sale_order_line_id_case_id=  [stack.case_id["sale_order_line_id"] if stack.case_id else -1 for idx, stack in enumerate(stacks_out) for event in stack]


df_frame_numbers = pd.DataFrame(data={"frame.number": frame_numbers, "sniff_time": sniff_time, "stack_idx": stack_numbers, "sale_order_id": sale_order_ids,"sale_order_line_id": sale_order_line_ids,"sale_order_line_id_case_id":sale_order_line_id_case_id,"purchase_requisition_id": purchase_requisition_ids,"purchase_requisition_line_id": purchase_requisition_line_ids, "purchase_order_id":purchase_order_ids, "case_id": case_id})

# Merge Activity Name from ground truth frame to event sequences for evaluation
merged_df = df_frame_numbers.merge(df_gt[["activity_name","start","bp_id"]], how="left",left_on="frame.number", right_on="start").drop(columns="start")

merged_df[["activity_name","bp_id"]] = merged_df.groupby("stack_idx")[["activity_name","bp_id"]].ffill()
#merged_df["activity_name"] = merged_df.groupby("stack_idx")["bp_id"].ffill()

# Merge with filtered interleaved test data
merged_df = df_test.merge(merged_df, on="frame.number")

In [96]:
unique_no_nan = lambda x: list(filter(None, pd.unique(x)))
first_unique = lambda x: unique_no_nan(x)[0]

def compare_values(x,y):
    # Multi index and casting magic - I just want to compare the bp_ids lol
    x = int(x[0])
    y = int(y[0])

    return x == y


res = merged_df.groupby("stack_idx").agg(sale_order_id = ("sale_order_id", unique_no_nan),sale_order_line_id=("sale_order_line_id", unique_no_nan), sale_order_line_id_case_id=("sale_order_line_id_case_id", unique_no_nan),purchase_requisition_id=("purchase_requisition_id", unique_no_nan),purchase_requisition_line_id=("purchase_requisition_line_id", unique_no_nan),purchase_order_id=("purchase_order_id",unique_no_nan),case_id=("case_id", first_unique),bp_id=("bp_id", unique_no_nan),frame_number_min=("frame.number","min"),frame_number_max =  ("frame.number","max"),sniff_time_min=("sniff_time_x","min"),sniff_time_max=("sniff_time_x","min"), activity_name=("activity_name", lambda x: x.head(1)))

res = res.merge(eval_df[["start_pred_true","end_pred_true","start_end_true"]], left_index=True, right_index=True)
res["stack_prediction"] = stack_predictions
# Apply the custom function to compare 'sale_order_line_id' and 'sale_order_line_id_case_id'
res["bp_true"] = res.apply(lambda x: compare_values(x["sale_order_line_id_case_id"], x["bp_id"]), axis = 1)
res["activity_true"] = res["activity_name"] ==  res["stack_prediction"]
#res.loc["Mean","bp_true"] = res["bp_true"].mean()
#res.loc["Mean","activity_true"] = res["activity_true"].mean()

In [97]:
print("------------------ Activity Type --------------")
print(classification_report(res["activity_name"], res["stack_prediction"]))

------------------ Activity Type --------------
                       precision    recall  f1-score   support

         BidSelection       0.89      0.80      0.84        10
  CreateCallForTender       1.00      0.80      0.89        10
  CreatePurchaseOrder       0.88      0.88      0.88         8
CreatePurchaseRequest       1.00      1.00      1.00         9
            CreateRfq       0.57      0.80      0.67        10
         ReceiveGoods       1.00      0.75      0.86         8
        SubmitPayment       0.67      0.75      0.71         8

             accuracy                           0.83        63
            macro avg       0.86      0.83      0.83        63
         weighted avg       0.86      0.83      0.83        63


In [95]:
first_int = lambda x: int(x[0])

pred = res["sale_order_line_id_case_id"].map(first_int)
true = res["bp_id"].map(first_int)

print("------------------ Activity Type --------------")
print(classification_report(true[pred!= -1], pred[pred!= -1], zero_division=0.0))

------------------ Activity Type --------------
              precision    recall  f1-score   support

         399       1.00      0.71      0.83         7
         400       1.00      1.00      1.00         6
         401       1.00      0.80      0.89         5
         402       1.00      1.00      1.00         4
         403       0.80      0.80      0.80         5
         404       0.46      0.86      0.60         7
         405       0.60      1.00      0.75         3
         406       0.00      0.00      0.00         5
         407       0.83      0.83      0.83         6
         408       1.00      1.00      1.00         6

    accuracy                           0.80        54
   macro avg       0.77      0.80      0.77        54
weighted avg       0.78      0.80      0.77        54


In [77]:
out = res.sort_values(by= "sniff_time_min")[["sniff_time_min","stack_prediction","case_id"]].reset_index(drop=True)
out.columns = ["timestamp", "activity", "case_id"]

In [78]:
out.to_csv("../../data_v3/out/ptp_xes_out.csv", index = False)